In [52]:
import torch, torchvision
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
import time

In [53]:
# Training settings
bs = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Training using " + device)

Training using cuda


In [55]:
# Load MNIST dataset
mnist_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = torchvision.datasets.MNIST(root='./data/mnist/', train=True,
                                          download=True,
                                          transform=mnist_transforms)

test_dataset = torchvision.datasets.MNIST(root='./data/mnist/', train=False,
                                         download=True,
                                         transform=mnist_transforms)


train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True)

In [43]:
# Build the model
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 520)
        self.fc2 = nn.Linear(520, 320)
        self.fc3 = nn.Linear(320, 240)
        self.fc4 = nn.Linear(240, 120)
        self.fc5 = nn.Linear(120, 10)
        
    def forward(self, x):
        x = x.view(-1, 784) # Flatten the data (n, 1, 28, 28) -> (n, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return x

In [45]:
model = Model()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=.5)

In [46]:
model.parameters

<bound method Module.parameters of Model(
  (fc1): Linear(in_features=784, out_features=520, bias=True)
  (fc2): Linear(in_features=520, out_features=320, bias=True)
  (fc3): Linear(in_features=320, out_features=240, bias=True)
  (fc4): Linear(in_features=240, out_features=120, bias=True)
  (fc5): Linear(in_features=120, out_features=10, bias=True)
)>

In [47]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move data to 'device'
        data, target = data.to(device), target.to(device)
        out = model(data)
        loss = criterion(out, target)
        
        # update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print the loss
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [48]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        out = model(data)
        
        # total loss
        test_loss += criterion(out, target).item()
        
        # get the index of the max value
        pred = out.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [49]:
# can be scripted
def run():
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Traing Time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testin Time: {m:.0f}m {s:.0f}s')
        
    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

In [56]:
run()

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.301912
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.302690
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.301332
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.298756
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.303226
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.298783
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.298901
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.297720
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.296524
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.298633
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.302532
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.300360
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.299579
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.298968
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.300301
Train Epoch: 1 | Batch Status: 9600/60

Train Epoch: 2 | Batch Status: 20480/60000 (34%) | Loss: 0.368322
Train Epoch: 2 | Batch Status: 21120/60000 (35%) | Loss: 0.454234
Train Epoch: 2 | Batch Status: 21760/60000 (36%) | Loss: 0.315324
Train Epoch: 2 | Batch Status: 22400/60000 (37%) | Loss: 0.286559
Train Epoch: 2 | Batch Status: 23040/60000 (38%) | Loss: 0.366613
Train Epoch: 2 | Batch Status: 23680/60000 (39%) | Loss: 0.339035
Train Epoch: 2 | Batch Status: 24320/60000 (41%) | Loss: 0.176133
Train Epoch: 2 | Batch Status: 24960/60000 (42%) | Loss: 0.278668
Train Epoch: 2 | Batch Status: 25600/60000 (43%) | Loss: 0.395727
Train Epoch: 2 | Batch Status: 26240/60000 (44%) | Loss: 0.397692
Train Epoch: 2 | Batch Status: 26880/60000 (45%) | Loss: 0.354529
Train Epoch: 2 | Batch Status: 27520/60000 (46%) | Loss: 0.454115
Train Epoch: 2 | Batch Status: 28160/60000 (47%) | Loss: 0.232948
Train Epoch: 2 | Batch Status: 28800/60000 (48%) | Loss: 0.560964
Train Epoch: 2 | Batch Status: 29440/60000 (49%) | Loss: 0.221999
Train Epoc

Train Epoch: 3 | Batch Status: 40320/60000 (67%) | Loss: 0.271448
Train Epoch: 3 | Batch Status: 40960/60000 (68%) | Loss: 0.139497
Train Epoch: 3 | Batch Status: 41600/60000 (69%) | Loss: 0.322706
Train Epoch: 3 | Batch Status: 42240/60000 (70%) | Loss: 0.275745
Train Epoch: 3 | Batch Status: 42880/60000 (71%) | Loss: 0.258020
Train Epoch: 3 | Batch Status: 43520/60000 (72%) | Loss: 0.089986
Train Epoch: 3 | Batch Status: 44160/60000 (74%) | Loss: 0.168351
Train Epoch: 3 | Batch Status: 44800/60000 (75%) | Loss: 0.072729
Train Epoch: 3 | Batch Status: 45440/60000 (76%) | Loss: 0.089390
Train Epoch: 3 | Batch Status: 46080/60000 (77%) | Loss: 0.273722
Train Epoch: 3 | Batch Status: 46720/60000 (78%) | Loss: 0.104461
Train Epoch: 3 | Batch Status: 47360/60000 (79%) | Loss: 0.090522
Train Epoch: 3 | Batch Status: 48000/60000 (80%) | Loss: 0.155150
Train Epoch: 3 | Batch Status: 48640/60000 (81%) | Loss: 0.272842
Train Epoch: 3 | Batch Status: 49280/60000 (82%) | Loss: 0.156282
Train Epoc

Train Epoch: 4 | Batch Status: 58880/60000 (98%) | Loss: 0.187140
Train Epoch: 4 | Batch Status: 59520/60000 (99%) | Loss: 0.242720
Traing Time: 0m 9s
Test set: Average loss: 0.0020, Accuracy: 9605/10000 (96%)
Testin Time: 0m 11s
Train Epoch: 5 | Batch Status: 0/60000 (0%) | Loss: 0.143539
Train Epoch: 5 | Batch Status: 640/60000 (1%) | Loss: 0.068307
Train Epoch: 5 | Batch Status: 1280/60000 (2%) | Loss: 0.139201
Train Epoch: 5 | Batch Status: 1920/60000 (3%) | Loss: 0.130267
Train Epoch: 5 | Batch Status: 2560/60000 (4%) | Loss: 0.040105
Train Epoch: 5 | Batch Status: 3200/60000 (5%) | Loss: 0.069463
Train Epoch: 5 | Batch Status: 3840/60000 (6%) | Loss: 0.111447
Train Epoch: 5 | Batch Status: 4480/60000 (7%) | Loss: 0.333610
Train Epoch: 5 | Batch Status: 5120/60000 (9%) | Loss: 0.016146
Train Epoch: 5 | Batch Status: 5760/60000 (10%) | Loss: 0.075571
Train Epoch: 5 | Batch Status: 6400/60000 (11%) | Loss: 0.067642
Train Epoch: 5 | Batch Status: 7040/60000 (12%) | Loss: 0.045055
Tra

Train Epoch: 6 | Batch Status: 17280/60000 (29%) | Loss: 0.094158
Train Epoch: 6 | Batch Status: 17920/60000 (30%) | Loss: 0.271703
Train Epoch: 6 | Batch Status: 18560/60000 (31%) | Loss: 0.056105
Train Epoch: 6 | Batch Status: 19200/60000 (32%) | Loss: 0.131657
Train Epoch: 6 | Batch Status: 19840/60000 (33%) | Loss: 0.064131
Train Epoch: 6 | Batch Status: 20480/60000 (34%) | Loss: 0.055259
Train Epoch: 6 | Batch Status: 21120/60000 (35%) | Loss: 0.046405
Train Epoch: 6 | Batch Status: 21760/60000 (36%) | Loss: 0.056682
Train Epoch: 6 | Batch Status: 22400/60000 (37%) | Loss: 0.048834
Train Epoch: 6 | Batch Status: 23040/60000 (38%) | Loss: 0.187896
Train Epoch: 6 | Batch Status: 23680/60000 (39%) | Loss: 0.051214
Train Epoch: 6 | Batch Status: 24320/60000 (41%) | Loss: 0.085987
Train Epoch: 6 | Batch Status: 24960/60000 (42%) | Loss: 0.018537
Train Epoch: 6 | Batch Status: 25600/60000 (43%) | Loss: 0.067679
Train Epoch: 6 | Batch Status: 26240/60000 (44%) | Loss: 0.085676
Train Epoc

Train Epoch: 7 | Batch Status: 36480/60000 (61%) | Loss: 0.021161
Train Epoch: 7 | Batch Status: 37120/60000 (62%) | Loss: 0.027162
Train Epoch: 7 | Batch Status: 37760/60000 (63%) | Loss: 0.012999
Train Epoch: 7 | Batch Status: 38400/60000 (64%) | Loss: 0.030977
Train Epoch: 7 | Batch Status: 39040/60000 (65%) | Loss: 0.019998
Train Epoch: 7 | Batch Status: 39680/60000 (66%) | Loss: 0.060721
Train Epoch: 7 | Batch Status: 40320/60000 (67%) | Loss: 0.029252
Train Epoch: 7 | Batch Status: 40960/60000 (68%) | Loss: 0.115160
Train Epoch: 7 | Batch Status: 41600/60000 (69%) | Loss: 0.096017
Train Epoch: 7 | Batch Status: 42240/60000 (70%) | Loss: 0.015080
Train Epoch: 7 | Batch Status: 42880/60000 (71%) | Loss: 0.080405
Train Epoch: 7 | Batch Status: 43520/60000 (72%) | Loss: 0.009253
Train Epoch: 7 | Batch Status: 44160/60000 (74%) | Loss: 0.053903
Train Epoch: 7 | Batch Status: 44800/60000 (75%) | Loss: 0.099349
Train Epoch: 7 | Batch Status: 45440/60000 (76%) | Loss: 0.095191
Train Epoc

Train Epoch: 8 | Batch Status: 55680/60000 (93%) | Loss: 0.109650
Train Epoch: 8 | Batch Status: 56320/60000 (94%) | Loss: 0.163040
Train Epoch: 8 | Batch Status: 56960/60000 (95%) | Loss: 0.023714
Train Epoch: 8 | Batch Status: 57600/60000 (96%) | Loss: 0.033415
Train Epoch: 8 | Batch Status: 58240/60000 (97%) | Loss: 0.027547
Train Epoch: 8 | Batch Status: 58880/60000 (98%) | Loss: 0.028434
Train Epoch: 8 | Batch Status: 59520/60000 (99%) | Loss: 0.035827
Traing Time: 0m 8s
Test set: Average loss: 0.0012, Accuracy: 9762/10000 (97%)
Testin Time: 0m 10s
Train Epoch: 9 | Batch Status: 0/60000 (0%) | Loss: 0.020910
Train Epoch: 9 | Batch Status: 640/60000 (1%) | Loss: 0.025830
Train Epoch: 9 | Batch Status: 1280/60000 (2%) | Loss: 0.035895
Train Epoch: 9 | Batch Status: 1920/60000 (3%) | Loss: 0.019609
Train Epoch: 9 | Batch Status: 2560/60000 (4%) | Loss: 0.038771
Train Epoch: 9 | Batch Status: 3200/60000 (5%) | Loss: 0.009889
Train Epoch: 9 | Batch Status: 3840/60000 (6%) | Loss: 0.019